# Przekształcanie układów wspoółrzędnych

Data i grupa:

Uczestnicy:

In [2]:
#All imports go here
import sympy as sym
import numpy as np
import matplotlib.pyplot as plt
import pickle
from typing import Optional, Tuple, Dict

## Zadanie 1.

Z wykorzystaniem biblioteki SymPy (https://docs.sympy.org/latest/index.html) wyprowadzić macierz rotacji na podstawie kątów Eulera dla prawoskrętnego układu współrzędnych przyjmując kolejność obrotów roll-pitch-yaw ($\alpha,\;\beta,\;\gamma$) oraz wyliczyć wartości liczbowe dla kątów $\alpha=95^o,\;\beta=88^o,\;\gamma=48^o$.
Następnie zaproponować metodę odwrotną tj. wyznaczyć kąty roll, pitch i yaw z  macierzy obrotu oraz sprawdzić jaki błąd powstał po wykonaniu przekształcenia w obu kierunkach. Skomentować uzyskane wyniki.
Macierze obrotu względem osi $x,y,z$:

\begin{equation*}
R_x =\begin{bmatrix} 
      1 && 0 && 0  \\ 
      0 && \cos \: \alpha && -\sin \: \alpha  \\
      0 && \sin \: \alpha && \cos \: \alpha  \\
   \end{bmatrix}
\end{equation*}

\begin{equation*}
R_y =\begin{bmatrix} 
      \cos \: \beta && 0 && \sin \: \beta  \\ 
      0 && 1 && 0  \\
      -\sin \: \beta && 0 && \cos \: \beta  \\
   \end{bmatrix}
\end{equation*}

\begin{equation*}
R_z =\begin{bmatrix}
      \cos \: \gamma && -\sin \: \gamma && 0\\
      \sin \: \gamma && \cos \: \gamma  && 0\\
      0 && 0 && 1  \\ 
   \end{bmatrix}
\end{equation*}

Przykład wykorzystania biblioteki SymPy:
```python
import sympy as sym
x, y, z, t = sym.symbols('x y z t')
K = sym.Matrix([[x*t,y/t],[z, 0]])
```


### Rozwiązanie

In [ ]:
# Your solution goes here

## Zadanie 2.

Wykorzystując dane z popularnego symulatora CARLA (http://carla.org/) wyrysować 3D Bounding Box na obrazie z kamery.
3D Bounding Box są zapisane w formacie:
* położenie obiektu (x,y,z) w układzie współrzędnych samochodu
* połowa wysokości
* połowa szerokość
* połowa długości

Dodatkowo posiadamy informację o pozycji i orientacji samochodu względem układu współrzędnych świata oraz 
pozycji i orientacji kamery względem układu współrzędnych samochodu.
Szczegółowe informacje znajdują się na stronie (https://carla.readthedocs.io/en/latest/)

Należy przekształcić punkty Bounding Box'a do układu współrzędnych świata, potem do układu współrzędnych kamery, następnie zrzutować je na płaszczyznę obrazu według następującego wzoru:
\begin{equation}
\begin{bmatrix}
u\\
v\\
1
\end{bmatrix}
= \frac{1}{z}
\begin{bmatrix} 
      f && 0 && u_0  \\ 
      0 && f && v_0  \\
      0 && 0 && 1  \\
\end{bmatrix}
\begin{bmatrix} 
      x\\
      y\\
      z
\end{bmatrix}
\quad z>0
\end{equation}
Gdzie:
* $u,v$ współrzedne obrazu
* $f$ ogniskowa kamery
* $u_0, v_0$ współrzedne środka obrazu.
* $x,y,z$ punkty w układzie współrzędnym kamery

Ostatecznie należy wyświetlić zdjęcie z wyrysowanymi liniami.
Wszystkie potrzebne dane wraz z obrazem znajdują się w słowniku w plikach "frame*.dat".

Opis danych:
* Zdjęcie: 
         data['sensors']['FrontCamera']
* Lista samochodów: 
         data['actors']
* Zarówno kamera oraz samochody mają następuące pola:
         ['transform']['location']['x']
         ['transform']['location']['y']
         ['transform']['location']['z']
         ['transform']['rotation']['roll']
         ['transform']['rotation']['pitch']
         ['transform']['rotation']['yaw']
* Samochód ma dodatkowo pola:
         ['bounding_box']['location']['x']
         ['bounding_box']['location']['y']
         ['bounding_box']['location']['z']
         ['bounding_box']['extent']['x']
         ['bounding_box']['extent']['y']
         ['bounding_box']['extent']['z']
* Kamera ma dodatkowo pola:
         ['raw_data'] zdjęcie w formacie BGR 0-255
         ['calibration']['intrinsic'] macierz 3x3 z prametrami inrinsic

         
<img src="drawing_transformations.png" width="600" />

<img src="lines.png" />

### Podpowiedzi:
* W przypadku CARLI macierz rotacji ma nastepującą formę:

\begin{equation}
\begin{bmatrix} 
      cos\beta cos\gamma  && sin\alpha sin\beta cos\gamma - sin\gamma cos\alpha && - sin\alpha sin\beta cos\gamma - sin\alpha sin\gamma     \\ 
      cos\beta sin\gamma && sin\alpha sin\beta sin\gamma + cos\alpha cos\gamma  && -cos\alpha sin\beta sin\gamma + sin\alpha cos\gamma \\
      sin\beta && - sin\alpha cos\beta && cos\alpha cos\beta  
\end{bmatrix}
\end{equation}
Czy jest to macierz dla układu lewoskrętnego czy dla układu prawoskrętnego?


* Przed rzutowaniem na kamerę  trzeba zamienić współrzędne w następujący sposób: $x=y, y=-z, z=x$. Dlaczego? 
* Dodatkowo predefiniowane funkcje

In [3]:
# Przykład wczytania danych
with open("data/frame1.dat", 'rb') as f:
    data = pickle.load(f)

In [4]:
# Przykład stworzenia 8 wierzchołków prostopadłościanu
def create_bb_points(actor: dict, scale: Optional[float] = 1) -> np.ndarray:
    cords = np.zeros((8, 4))
    bb = actor["bounding_box"]
    extent_x = bb["extent"]["x"]
    extent_y = bb["extent"]["y"]
    extent_z = bb["extent"]["z"]
    center = np.array([bb["location"]["x"], bb["location"]["y"], bb["location"]["z"], 0])
    # base
    cords[0, :] = np.array([scale * extent_x, scale * extent_y, -scale * extent_z, 1])
    cords[1, :] = np.array([-scale * extent_x, scale * extent_y, -scale * extent_z, 1])
    cords[2, :] = np.array([-scale * extent_x, -scale * extent_y, -scale * extent_z, 1])
    cords[3, :] = np.array([scale * extent_x, -scale * extent_y, -scale * extent_z, 1])
    # top
    cords[4, :] = np.array([scale * extent_x, scale * extent_y, scale * extent_z, 1])
    cords[5, :] = np.array([-scale * extent_x, scale * extent_y, scale * extent_z, 1])
    cords[6, :] = np.array([-scale * extent_x, -scale * extent_y, scale * extent_z, 1])
    cords[7, :] = np.array([scale * extent_x, -scale * extent_y, scale * extent_z, 1])
    return (cords+center).T

In [3]:
# Wyrysowanie BB na obrazie 
def draw_bbox3d(points, color: Optional[str] = 'r'):
    x, y = only_objects_in_image(points)
    # draw lines
    if x.size > 0:
        # base
        plt.plot((x[0], x[1]), (y[0], y[1]), color=color)
        plt.plot((x[1], x[2]), (y[1], y[2]), color=color)
        plt.plot((x[2], x[3]), (y[2], y[3]), color=color)
        plt.plot((x[3], x[0]), (y[3], y[0]), color=color)
        # top
        plt.plot((x[4], x[5]), (y[4], y[5]), color=color)
        plt.plot((x[5], x[6]), (y[5], y[6]), color=color)
        plt.plot((x[6], x[7]), (y[6], y[7]), color=color)
        plt.plot((x[7], x[4]), (y[7], y[4]), color=color)
        # base-top
        plt.plot((x[0], x[4]), (y[0], y[4]), color=color)
        plt.plot((x[1], x[5]), (y[1], y[5]), color=color)
        plt.plot((x[2], x[6]), (y[2], y[6]), color=color)
        plt.plot((x[3], x[7]), (y[3], y[7]), color=color)


def only_objects_in_image(points: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    idx = points[:, 2] > 0
    x = points[idx, 0]
    y = points[idx, 1]
    idx_x = np.logical_and(x > 0, x < 1242)
    idx_y = np.logical_and(y > 0, y < 352)
    idx_size = np.logical_and(idx_y, idx_x)
    if np.sum(idx_size) > 0:
        return points[:, 0], points[:, 1]
    else:
        return x[idx_size], y[idx_size]

### Rozwiązanie

In [5]:
# Your solution goes here